In [ ]:
pip install mljar-supervised

In [ ]:
import numpy as np
import pandas as pd
import missingno as msno

import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import ExtraTreesClassifier, VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, accuracy_score
import graphviz
from sklearn.tree import DecisionTreeClassifier, export_graphviz

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2021/test.csv')
final = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2021/sample_submission.csv')
train.head()

# Understanding Data

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
missing_percentages = (train[train.columns].isnull().sum() / train.shape[0]) * 100
missing_percentages

In [ ]:
missing_percentages_test = (test[test.columns].isnull().sum() / test.shape[0]) * 100
missing_percentages_test

### The columns with missing values are-

* Age
* Ticket
* Fare
* Cabin
* Embarked

# EDA

In [ ]:
for i in train.columns:
    print("The number of unique values in {} is {}".format(i, len(train[i].unique())))

### The categorical features are
* Pclass
* Sex
* Embarked
* Parch
* SibSp

### The continuous features are
* Age
* Fare

### The ones which will be dealt manually are
* Ticket
* Cabin
* Name

In [ ]:
categorical_features = ["Pclass","Sex","Embarked","Parch","SibSp"]
continuous_features = ["Age","Fare"]

## Checking the count & distribution of Survived


In [ ]:
fig = plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
ax = sns.countplot(x="Survived",data=train)
plt.subplot(1,2,2)
sns.distplot(train.loc[: ,'Survived'], hist_kws={"color":"r"}, kde_kws={"color":"b", "lw":2})
plt.show()

## Pclass

In [ ]:
fig = plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
ax = sns.countplot(x="Pclass",data=train)
plt.subplot(1,2,2)
sns.distplot(train.loc[: ,"Pclass"], hist_kws={"color":"r"}, kde_kws={"color":"b", "lw":2})
plt.show()

## Parch

In [ ]:
fig = plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
ax = sns.countplot(x="Parch",data=train)
plt.subplot(1,2,2)
sns.distplot(train.loc[: ,"Parch"], hist_kws={"color":"r"}, kde_kws={"color":"b", "lw":2})
plt.show()

## SibSp

In [ ]:
fig = plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
ax = sns.countplot(x="SibSp",data=train)
plt.subplot(1,2,2)
sns.distplot(train.loc[: ,"SibSp"], hist_kws={"color":"r"}, kde_kws={"color":"b", "lw":2})
plt.show()

## Sex

In [ ]:
fig = plt.figure(figsize=(6,6))
sns.set_palette(["#8072fa","orange"])
ax = sns.countplot(x="Sex",data=train)
plt.show()

## Embarked

In [ ]:
fig = plt.figure(figsize=(6,6))
sns.set_palette(["#8072fa","orange","Red"])
ax = sns.countplot(x="Embarked",data=train)
plt.show()

# Data Preprocessing

Handling missing data

In [ ]:
train['Age'].fillna(train['Age'].mean(),inplace=True)
test['Age'].fillna(train['Age'].mean(),inplace=True)

train['Fare'].fillna(train['Fare'].mean(),inplace=True)
test['Fare'].fillna(train['Fare'].mean(),inplace=True)

train['Embarked'].fillna(train['Embarked'].mode()[0],inplace=True)
test['Embarked'].fillna(train['Embarked'].mode()[0],inplace=True)

Ticket number,Cabin & Name number doesn't seem to influence Survival.

In [ ]:
train.drop(['Name','Ticket','Cabin','PassengerId'], axis=1, inplace=True)
test.drop(['Name','Ticket','Cabin','PassengerId'], axis=1, inplace=True)

Applying a log function to reduce the influence of outliers in Fare column. Since there are a large number of outliers, removing them will lead to loss of a large numberof points.

In [ ]:
train['Fare'] = train['Fare'].map(lambda i: np.log(i) if i > 0 else 0)
test['Fare'] = test['Fare'].map(lambda i: np.log(i) if i > 0 else 0)

Adding Parch and SibSp into one feature

In [ ]:
train["relatives"] = train["Parch"] + train["SibSp"] + 1
test["relatives"] = test["Parch"] + test["SibSp"] + 1

# Modelling

Label Encoding the categorical features

In [ ]:
object_cols = ['Sex','Embarked']
for col in object_cols:
    label_encoder = LabelEncoder()
    label_encoder.fit(train[col])
    train[col] = label_encoder.transform(train[col])
    test[col] = label_encoder.transform(test[col])

In [ ]:
features = ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked','relatives']
target = train['Survived'].values

## Logistic Regression

In [ ]:
lr = LogisticRegression()
lr.fit(train[features], target)
print("Logistic Regression ROC AUC score:", roc_auc_score(target, lr.predict_proba(train[features])[:,1]))
print('Logistic Regression Accuracy score:', accuracy_score(target, lr.predict(train[features])))

In [ ]:
final['Survived'] = lr.predict(test[features])
final.to_csv('LR.csv',index=False)

## XGBoost

In [ ]:
xgb = XGBClassifier()
xgb.fit(train[features], target)
print("XGB ROC AUC score:", roc_auc_score(target, xgb.predict_proba(train[features])[:,1]))
print('XGB Accuracy score:', accuracy_score(target, xgb.predict(train[features])))

In [ ]:
final['Survived'] = xgb.predict(test[features])
final.to_csv('XGB.csv',index=False)

# AutoML

In [ ]:
from supervised.automl import AutoML
automl = AutoML(eval_metric="accuracy")
automl.fit(train[features], target)
automl.report()

In [ ]:
final['Survived'] = automl.predict(test[features])
final.to_csv('AutoML.csv',index=False)

# Decision Tree

In [ ]:
dt = DecisionTreeClassifier(
    max_depth=4,
    min_samples_leaf=2)
dt.fit(train[features], target)

In [ ]:
dot_data = export_graphviz(
    dt,
    out_file=None,
    feature_names=train[features].columns,
    class_names=['0', '1'],
    filled=True,
    rounded=False,
    special_characters=True,
    precision=3
)
graph = graphviz.Source(dot_data)
graph 

In [ ]:
y_pred = dt.predict(test[features]).astype(int)
final['Survived'] = y_pred
final.to_csv("DT.csv", index=False)

# LGBM

In [ ]:
lgbm = LGBMClassifier(boosting_type = 'dart',num_leaves = 32,max_depth = 10,colsample_bytree = 0.8,extra_trees = True,n_jobs = -1,random_state = 42)
lgbm.fit(train[features], target)

In [ ]:
y_pred = lgbm.predict(test[features]).astype(int)
final['Survived'] = y_pred
final.to_csv("LGBM.csv", index=False)

# Extra Classifier

In [ ]:
ext = ExtraTreesClassifier(n_estimators = 1000,max_depth = 17,min_samples_split = 25,min_samples_leaf = 18,n_jobs = -1,random_state = 42)
ext.fit(train[features], target)

In [ ]:
y_pred = ext.predict(test[features]).astype(int)
final['Survived'] = y_pred
final.to_csv("EXT.csv", index=False)

# Gradient Boosting

In [ ]:
gb=GradientBoostingClassifier(max_depth= 2, n_estimators = 400)
gb.fit(train[features], target)

In [ ]:
y_pred = gb.predict(test[features]).astype(int)
final['Survived'] = y_pred
final.to_csv("GB.csv", index=False)

# Voting CLassifier

In [ ]:
clf = VotingClassifier(estimators=[('DT',dt),('EXT',ext),('LGBM' , lgbm) , ('GB',gb)], voting='soft')
clf.fit(train[features], target)

In [ ]:
y_pred = clf.predict(test[features]).astype(int)
final['Survived'] = y_pred
final.to_csv("VC.csv", index=False)